# Backtesting ML Classification-Based

In [ ]:
???

![](src/res_classification.png)

## Load the model

In [ ]:
import pickle

In [ ]:
with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [ ]:
model_dt

## Load the data

In [ ]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

## Backtesting.py Library

### Create your Strategy Class

In [ ]:
from backtesting import Backtest, Strategy

In [ ]:
Backtest(???)

In [ ]:
df_explanatory = df.drop(columns=['change_tomorrow', 'change_tomorrow_direction'])

In [ ]:
model_dt.predict(X=df_explanatory)

In [ ]:
explanatory_today = df_explanatory.iloc[[-1],:]

In [ ]:
forecast_tomorrow = model_dt.predict(explanatory_today)[0]

In [ ]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = model_dt

    def next(self):
        explanatory_today = df_explanatory.iloc[[-1],:]
        forecast_tomorrow = model_dt.predict(explanatory_today)[0]
        
        # conditions to sell or buy

#### Calculate Purchase Recommendation

##### Buy if it goes up

In [ ]:
list_buy_sell = []

In [ ]:
for tomorrow_direction in df.change_tomorrow_direction:
  if tomorrow_direction == 'UP':
    list_buy_sell.append(1)
  elif tomorrow_direction == 'DOWN':
    list_buy_sell.append(-1)

In [ ]:
df['buy_sell'] = list_buy_sell

In [ ]:
df[['change_tomorrow_direction', 'buy_sell']].head(10)

##### Buy if it goes and sell if down

> You can only sell if you have already bought

In [ ]:
list_buy_sell = []
already_bought = False

In [ ]:
for tomorrow_direction in df.change_tomorrow_direction:
  if tomorrow_direction == 'UP' and already_bought == False:
    list_buy_sell.append(1)
    already_bought = True
  elif tomorrow_direction == 'DOWN' and already_bought == True:
    list_buy_sell.append(-1)
    already_bought = False
  else:
    list_buy_sell.append(0)

In [ ]:
df['buy_sell_track'] = list_buy_sell

In [ ]:
df[['change_tomorrow_direction', 'buy_sell', 'buy_sell_track']].head(10)

#### Add conditions to the strategy

In [ ]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1],:]
        forecast_tomorrow = model_dt.predict(explanatory_today)[0]
        
        # conditions to sell or buy
        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [ ]:
bt = Backtest(data = df_explanatory, ???,
              cash=???, commission=.002, exclusive_orders=True)

### Run backtesting

### Interpret backtesting results

In [ ]:
results.to_frame(name='Values').loc[:'Return [%]']

In [ ]:
df.Close.plot()

Classification Model | Regression Model
-|-
![](src/pred_classification.png) | ![](src/pred_regression.png)

Classification Strategy | Regression Strategy
-|-
![](src/res_classification.png) | ![](src/res_regression.png)